# Boundary mapping

The purpose of this notebook is to mess around with all the different options available with boundary mapping

In [1]:
import os
import sys
from typing import Tuple
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


import database, gen_spectra
import testing_utils

from constants import WATER_MASS, SINGLY_CHARGED_B_BASE, SINGLY_CHARGED_Y_BASE
from utils import ppm_to_da, overlap_intervals
from preprocessing import merge_search, preprocessing_utils
from collections import defaultdict
from objects import Spectrum

#Assumptions:
max_peptide_length = 20
ppm_tolerance = 20

import matplotlib.pyplot as plt

In [2]:
datasets = testing_utils.define_data()

dataset = datasets[0]

input_spectra_path = dataset[0]
input_spectra, boundaries, mz_mapping = testing_utils.preprocess_input_spectra(input_spectra_path, ppm_tolerance)


correct_sequences = testing_utils.generate_truth_set(datasets[0])

path = dataset[2]
db = database.build(path)

Loading spectra...
Done


In [3]:
matched_masses_b, matched_masses_y, db = testing_utils.modified_match_masses(boundaries, db, max_peptide_length)
print('Finished matching masses')

On batch 1/1
On protein 279/279 [100%]
Sorting the set of protein masses...
Done
Finished matching masses


# Grabbing hits

In [4]:
print('Grabbing hits...')
spectrum_num = 0
correct_sequence = correct_sequences[spectrum_num]
print(correct_sequence)

input_spectrum = input_spectra[spectrum_num]

b_hits =[]
y_hits = []
b_set = set()
y_set = set()
correct_hits = []
mz_miss_set = set()
#Remember to add in abundance if it is helpful
input_num = 0
testing_utils.find_hits(mz_mapping, boundaries, input_spectrum, input_num, matched_masses_b, matched_masses_y, b_hits, y_hits, b_set, y_set, mz_miss_set)
testing_utils.append_correct_hits(correct_hits, correct_sequence, input_spectrum, ppm_tolerance)
print('Done')

Grabbing hits...
DPQVEQLEL
Done


# Writing hits

In [5]:
#Writing b and y hits
print('Writing data...')
with open("b_hits.txt", 'w') as b:
    for x in b_hits:
        pep_id = x[0]
        w = x[1][0]
        prot_id = x[1][1]
        seq = x[1][2]
        loc = x[1][3]
        ion = x[1][4]
        charge = x[1][5]
        out = [pep_id, w, prot_id, seq, loc, ion, charge]
        b.write('\t'.join([str(i) for i in out]) + '\n')
with open("y_hits.txt", 'w') as y_file:
    for y in y_hits:
        pep_id = y[0]
        w = y[1][0]
        prot_id = y[1][1]
        seq = y[1][2]
        loc = y[1][3]
        ion = y[1][4]
        charge = y[1][5]
        out = [pep_id, w, prot_id, seq, loc, ion, charge]
        y_file.write('\t'.join([str(i) for i in out]) + '\n')
print('Done')

Writing data...
Done


# Clustering b hits

In [14]:
# b_hits

import collections
import operator

Hit = collections.namedtuple('Hit', 'pid start end seq')

def write_cluster(cluster):
    if len(cluster) == 0 : return None
    O = []

    O.append(len(cluster))
    O.append(cluster[0].pid)

    max_len = 0
    max_hit = None

    for hit in cluster:
        l = hit.end - hit.start + 1
        if l > max_len:
            max_len = l
            max_hit = hit

    O.append(max_hit.seq)
    
    for hit in cluster:
        O.append( (hit.start, hit.end, hit.seq) )
    
#     print( '\t'.join( [str(o) for o in O] ) )
    with open('clusters.txt', 'a') as c:
        c.write( '\t'.join( [str(o) for o in O] ) )
        c.write('\n')




file_name = 'b_hits.txt'
ion = 'b'
# file_name = sys.argv[1]

hits = []

for l in open(file_name):
    A = l.rstrip().split('\t')
    pid = int(A[2])
    start = int(A[4].split('-')[0])
    end = int(A[4].split('-')[1])
    seq = A[3]

    hits.append( Hit(pid=pid, start=start, end=end, seq=seq) )

sorted_hits = sorted(hits, key=operator.attrgetter('pid', 'start', 'end'))

last_pid = None
last_start = None

cluster = []

with open('clusters.txt', 'w') as c:
    c.write('')

if ion == 'b':
    for hit in sorted_hits:
        if last_pid == hit.pid and last_start == hit.start:
            cluster.append(hit)
        else:
            write_cluster(cluster)
            cluster = [hit]
        last_pid = hit.pid
        last_start = hit.start

# Printing data

In [15]:
# b_hits
b_cluster_array = []
cluster = collections.namedtuple('cluster', 'score pid seq indices')
with open('clusters.txt', 'r') as c:
    for line in c:
        A = line.rstrip().split('\t')
        score = int(A[0])
        pid = int(A[1])
        seq = A[2]
        indices = []
        [indices.append(A[x]) for x in range(3,len(A))]

        b_cluster_array.append(cluster(score=score, pid=pid, seq=seq, indices=indices) )

b_sorted_clusters = sorted(b_cluster_array, key=operator.attrgetter('score', 'pid'), reverse = True)
for cluster in b_sorted_clusters:
    non_indices = str(cluster.score) + '\t' + str(cluster.pid) + '\t' + cluster.seq
    print(non_indices + '\t'+ '\t'.join([str(o) for o in cluster.indices]))


7	274	DPQVEQLE	(60, 62, 'DPQ')	(60, 63, 'DPQV')	(60, 64, 'DPQVE')	(60, 65, 'DPQVEQ')	(60, 65, 'DPQVEQ')	(60, 66, 'DPQVEQL')	(60, 67, 'DPQVEQLE')
5	245	IEKNTDGVNFYNIL	(260, 261, 'IE')	(260, 263, 'IEKN')	(260, 264, 'IEKNT')	(260, 271, 'IEKNTDGVNFYN')	(260, 273, 'IEKNTDGVNFYNIL')
5	152	KNDIPKDK	(78, 78, 'K')	(78, 79, 'KN')	(78, 80, 'KND')	(78, 84, 'KNDIPKD')	(78, 85, 'KNDIPKDK')
5	112	TQAGVEELDPENKIP	(57, 57, 'T')	(57, 60, 'TQAG')	(57, 62, 'TQAGVE')	(57, 63, 'TQAGVEE')	(57, 71, 'TQAGVEELDPENKIP')
5	106	FSLHDPL	(472, 473, 'FS')	(472, 475, 'FSLH')	(472, 477, 'FSLHDP')	(472, 477, 'FSLHDP')	(472, 478, 'FSLHDPL')
5	41	HSLMPMLE	(237, 240, 'HSLM')	(237, 242, 'HSLMPM')	(237, 242, 'HSLMPM')	(237, 243, 'HSLMPML')	(237, 244, 'HSLMPMLE')
4	269	ELDVEE	(172, 173, 'EL')	(172, 174, 'ELD')	(172, 176, 'ELDVE')	(172, 177, 'ELDVEE')
4	268	TDFTLTISSVQAEDL	(98, 98, 'T')	(98, 105, 'TDFTLTIS')	(98, 110, 'TDFTLTISSVQAE')	(98, 112, 'TDFTLTISSVQAEDL')
4	255	PNIDEKGQVCLPV	(77, 80, 'PNID')	(77, 81, 'PNIDE')	(77, 82, 

3	85	LEEDIIPE	(751, 752, 'LE')	(751, 757, 'LEEDIIP')	(751, 758, 'LEEDIIPE')
3	85	IPEEDII	(756, 759, 'IPEE')	(756, 761, 'IPEEDI')	(756, 762, 'IPEEDII')
3	85	YAAFNNRPPSTWLT	(1066, 1067, 'YA')	(1066, 1071, 'YAAFNN')	(1066, 1079, 'YAAFNNRPPSTWLT')
3	85	PDQQLYNVEATSYALLA	(1223, 1225, 'PDQ')	(1223, 1226, 'PDQQ')	(1223, 1239, 'PDQQLYNVEATSYALLA')
3	85	KLCHSE	(1504, 1504, 'K')	(1504, 1508, 'KLCHS')	(1504, 1509, 'KLCHSE')
3	83	TPAAAFR	(88, 88, 'T')	(88, 92, 'TPAAA')	(88, 94, 'TPAAAFR')
3	82	KIPVPEDKYT	(78, 78, 'K')	(78, 86, 'KIPVPEDKY')	(78, 87, 'KIPVPEDKYT')
3	79	KND	(140, 140, 'K')	(140, 141, 'KN')	(140, 142, 'KND')
3	76	KKCGWFHPPANE	(557, 557, 'K')	(557, 558, 'KK')	(557, 568, 'KKCGWFHPPANE')
3	76	KKSPETLKCR	(952, 952, 'K')	(952, 953, 'KK')	(952, 961, 'KKSPETLKCR')
3	76	LFPRGYFHCLSSQDIL	(1080, 1081, 'LF')	(1080, 1082, 'LFP')	(1080, 1095, 'LFPRGYFHCLSSQDIL')
3	76	QPARED	(1329, 1330, 'QP')	(1329, 1334, 'QPARED')	(1329, 1334, 'QPARED')
3	76	PQQPQPPPQQQAAPQ	(1653, 1654, 'PQ')	(1653, 1656, 'PQQP')

2	254	KVDHIKILVEDELS	(373, 373, 'K')	(373, 386, 'KVDHIKILVEDELS')
2	254	ELSSPVVV	(384, 385, 'EL')	(384, 391, 'ELSSPVVV')
2	254	VSNIAPAAV	(410, 414, 'VSNIA')	(410, 418, 'VSNIAPAAV')
2	254	QCGLTVIDLEV	(442, 448, 'QCGLTVI')	(442, 452, 'QCGLTVIDLEV')
2	254	GTCVRFSPLMTAEGLGTR	(454, 466, 'GTCVRFSPLMTAE')	(454, 471, 'GTCVRFSPLMTAEGLGTR')
2	254	KQEV	(500, 500, 'K')	(500, 503, 'KQEV')
2	254	KK	(550, 550, 'K')	(550, 551, 'KK')
2	254	KLNE	(551, 551, 'K')	(551, 554, 'KLNE')
2	254	LESDLTFKIGPEYK	(555, 556, 'LE')	(555, 568, 'LESDLTFKIGPEYK')
2	254	SFN	(662, 663, 'SF')	(662, 664, 'SFN')
2	254	KPFKRSLRG	(705, 705, 'K')	(705, 713, 'KPFKRSLRG')
2	254	KRSLRGSDAV	(708, 708, 'K')	(708, 717, 'KRSLRGSDAV')
2	254	IED	(726, 727, 'IE')	(726, 728, 'IED')
2	254	EDLEKVEQLS	(727, 729, 'EDL')	(727, 736, 'EDLEKVEQLS')
2	254	KVEQ	(731, 731, 'K')	(731, 734, 'KVEQ')
2	254	QLSSGLE	(734, 739, 'QLSSGL')	(734, 740, 'QLSSGLE')
2	254	LEAH	(744, 745, 'LE')	(744, 747, 'LEAH')
2	253	APGLQLLLRACEQ	(7, 9, 'APG')	(7, 19, 'APGLQLLLR

2	236	KK	(31, 31, 'K')	(31, 32, 'KK')
2	236	ELKDLLQTE	(33, 34, 'EL')	(33, 41, 'ELKDLLQTE')
2	236	KDADAVD	(50, 50, 'K')	(50, 56, 'KDADAVD')
2	236	ELD	(61, 62, 'EL')	(61, 63, 'ELD')
2	235	KPSPMPVSPECLE	(46, 46, 'K')	(46, 58, 'KPSPMPVSPECLE')
2	235	LLKELQDLALQGAKERA	(75, 87, 'LLKELQDLALQGA')	(75, 91, 'LLKELQDLALQGAKERA')
2	235	KERA	(88, 88, 'K')	(88, 91, 'KERA')
2	235	ERAQQPL	(89, 92, 'ERAQ')	(89, 95, 'ERAQQPL')
2	235	EQQHSS	(114, 119, 'EQQHSS')	(114, 119, 'EQQHSS')
2	235	KHRDAAAE	(136, 136, 'K')	(136, 143, 'KHRDAAAE')
2	235	RDAAAEVP	(138, 142, 'RDAAA')	(138, 145, 'RDAAAEVP')
2	235	TT	(167, 167, 'T')	(167, 168, 'TT')
2	235	EPNQ	(178, 180, 'EPN')	(178, 181, 'EPNQ')
2	235	TQSPTSL	(200, 200, 'T')	(200, 206, 'TQSPTSL')
2	235	QQQARKAK	(227, 233, 'QQQARKA')	(227, 234, 'QQQARKAK')
2	235	EEVPTAA	(255, 257, 'EEV')	(255, 261, 'EEVPTAA')
2	235	GKGEL	(301, 303, 'GKG')	(301, 305, 'GKGEL')
2	235	PQGGKGRE	(327, 328, 'PQ')	(327, 334, 'PQGGKGRE')
2	235	ELTAEKRLE	(367, 368, 'EL')	(367, 375, 'ELTAEKRLE')
2	

2	202	VLSANLHGGSLVVNYP	(664, 668, 'VLSAN')	(664, 679, 'VLSANLHGGSLVVNYP')
2	202	SKENSQMFQGRPCK	(707, 716, 'SKENSQMFQG')	(707, 720, 'SKENSQMFQGRPCK')
2	202	IELGCVKYPFEN	(760, 761, 'IE')	(760, 771, 'IELGCVKYPFEN')
2	202	ELGCVKYPFENELP	(761, 762, 'EL')	(761, 774, 'ELGCVKYPFENELP')
2	202	HPVTTYKAGD	(820, 821, 'HP')	(820, 829, 'HPVTTYKAGD')
2	202	TT	(823, 823, 'T')	(823, 824, 'TT')
2	202	TYKAG	(824, 824, 'T')	(824, 828, 'TYKAG')
2	202	KN	(853, 853, 'K')	(853, 854, 'KN')
2	202	TLVRSSA	(868, 868, 'T')	(868, 874, 'TLVRSSA')
2	202	KGRGHSTS	(882, 882, 'K')	(882, 889, 'KGRGHSTS')
2	202	TT	(954, 954, 'T')	(954, 955, 'TT')
2	202	LGQSVEYRH	(956, 960, 'LGQSV')	(956, 964, 'LGQSVEYRH')
2	202	PNISEPEEPKIRFVAGI	(974, 977, 'PNIS')	(974, 990, 'PNISEPEEPKIRFVAGI')
2	202	EEPKIRFVAGIHGNA	(980, 988, 'EEPKIRFVA')	(980, 994, 'EEPKIRFVAGIHGNA')
2	202	PSLNPDG	(1031, 1034, 'PSLN')	(1031, 1037, 'PSLNPDG')
2	202	KDLDTDFTSN	(1058, 1058, 'K')	(1058, 1067, 'KDLDTDFTSN')
2	202	NKETL	(1112, 1113, 'NK')	(1112, 1116, 'NKETL

...

1	27	QDP	(341, 343, 'QDP')
1	27	GEAGRLI	(346, 352, 'GEAGRLI')
1	27	MNSRSLELFSID	(359, 370, 'MNSRSLELFSID')
1	27	NSR	(360, 362, 'NSR')
1	27	LE	(364, 365, 'LE')
1	27	EL	(365, 366, 'EL')
1	27	FS	(367, 368, 'FS')
1	27	DPQ	(370, 372, 'DPQ')
1	27	PQ	(371, 372, 'PQ')
1	27	T	(378, 378, 'T')
1	27	ERHY	(388, 391, 'ERHY')
1	27	T	(408, 408, 'T')
1	27	T	(413, 413, 'T')
1	27	VADRNDHA	(414, 421, 'VADRNDHA')
1	27	RNDHAPVFEQAQ	(417, 428, 'RNDHAPVFEQAQ')
1	27	HAPVFEQAQY	(420, 429, 'HAPVFEQAQY')
1	27	T	(432, 432, 'T')
1	27	VEE	(437, 439, 'VEE')
1	27	EGYPILQLR	(439, 447, 'EGYPILQLR')
1	27	YPILQLRATD	(441, 450, 'YPILQLRATD')
1	27	ATDGDAPPNANL	(448, 459, 'ATDGDAPPNANL')
1	27	T	(449, 449, 'T')
1	27	GDAP	(451, 454, 'GDAP')
1	27	APPNA	(453, 457, 'APPNA')
1	27	RFVGSPA	(462, 468, 'RFVGSPA')
1	27	VGSPA	(464, 468, 'VGSPA')
1	27	T	(471, 471, 'T')
1	27	IDPRSGLIS	(479, 487, 'IDPRSGLIS')
1	27	RSGLIST	(482, 488, 'RSGLIST')
1	27	LISTSGR	(485, 491, 'LISTSGR')
1	27	ISTSGRVDREHMESYE	(486, 501, 'ISTSGRVDREHMESYE')
1	27	STSG

1	27	LSWCRAATLGHRAVP	(3020, 3034, 'LSWCRAATLGHRAVP')
1	27	CRAATLGH	(3023, 3030, 'CRAATLGH')
1	27	ATLGHRAVPAASYGRI	(3026, 3041, 'ATLGHRAVPAASYGRI')
1	27	T	(3027, 3027, 'T')
1	27	GHRAV	(3029, 3033, 'GHRAV')
1	27	SYGRIYA	(3037, 3043, 'SYGRIYA')
1	27	T	(3047, 3047, 'T')
1	27	QP	(3052, 3053, 'QP')
1	27	PASR	(3053, 3056, 'PASR')
1	27	RRQLSKERLEEVP	(3068, 3080, 'RRQLSKERLEEVP')
1	27	RQLS	(3069, 3072, 'RQLS')
1	27	QLSKE	(3070, 3074, 'QLSKE')
1	27	K	(3073, 3073, 'K')
1	27	RLEEVPV	(3075, 3081, 'RLEEVPV')
1	27	LE	(3076, 3077, 'LE')
1	27	HP	(3087, 3088, 'HP')
1	27	PLSRPGSQ	(3088, 3095, 'PLSRPGSQ')
1	27	SRPGS	(3090, 3094, 'SRPGS')
1	27	RPGSQER	(3091, 3097, 'RPGSQER')
1	27	PGSQERLDTAPARLE	(3092, 3106, 'PGSQERLDTAPARLE')
1	27	RLDTAPARLEARDR	(3097, 3110, 'RLDTAPARLEARDR')
1	27	ARLEAR	(3103, 3108, 'ARLEAR')
1	27	LE	(3105, 3106, 'LE')
1	27	RDRG	(3108, 3111, 'RDRG')
1	27	GSTLPRRQPPRDYP	(3111, 3124, 'GSTLPRRQPPRDYP')
1	27	STLPRRQPPRDYPG	(3112, 3125, 'STLPRRQPPRDYPG')
1	27	T	(3113, 3113, 'T')
1	27	QP	(3118

1	21	GVMEQLG	(46, 52, 'GVMEQLG')
1	21	ARPRVEYPAHQAMN	(54, 67, 'ARPRVEYPAHQAMN')
1	21	YPAH	(60, 63, 'YPAH')
1	21	VGPQS	(69, 73, 'VGPQS')
1	21	PQ	(71, 72, 'PQ')
1	21	IE	(74, 75, 'IE')
1	21	EGGAHEGLQ	(75, 83, 'EGGAHEGLQ')
1	21	LGPFGN	(85, 90, 'LGPFGN')
1	21	GPFGNI	(86, 91, 'GPFGNI')
1	21	EL	(97, 98, 'EL')
1	21	LTGDNIPKDFSEDQGYP	(98, 114, 'LTGDNIPKDFSEDQGYP')
1	21	T	(99, 99, 'T')
1	21	DNIP	(101, 104, 'DNIP')
1	21	NIPKDF	(102, 107, 'NIPKDF')
1	21	K	(105, 105, 'K')
1	21	FS	(107, 108, 'FS')
1	21	QGY	(111, 113, 'QGY')
1	21	NPCPLGKT	(118, 125, 'NPCPLGKT')
1	21	PCPLGKT	(119, 125, 'PCPLGKT')
1	21	LGKTAD	(122, 127, 'LGKTAD')
1	21	GKTA	(123, 126, 'GKTA')
1	21	K	(124, 124, 'K')
1	21	T	(125, 125, 'T')
1	21	LE	(131, 132, 'LE')
1	21	T	(137, 137, 'T')
1	21	FS	(140, 141, 'FS')
1	21	FDPEHDYPGL	(151, 160, 'FDPEHDYPGL')
1	21	DPEHDYPGLGKW	(152, 163, 'DPEHDYPGLGKW')
1	21	LGKW	(160, 163, 'LGKW')
1	21	K	(162, 162, 'K')
1	21	NK	(164, 165, 'NK')
1	21	K	(166, 166, 'K')
1	21	K	(179, 179, 'K')
1	21	RRSVNPYLQG	(180, 

1	15	K	(432, 432, 'K')
1	15	IE	(433, 434, 'IE')
1	15	T	(438, 438, 'T')
1	15	PDGTVVTTVTTV	(441, 452, 'PDGTVVTTVTTV')
1	15	T	(444, 444, 'T')
1	15	T	(448, 448, 'T')
1	15	T	(451, 451, 'T')
1	15	VQSRPRV	(452, 458, 'VQSRPRV')
1	15	VDGKLDS	(458, 464, 'VDGKLDS')
1	15	K	(461, 461, 'K')
1	15	K	(471, 471, 'K')
1	15	VEVTEKMTTVLSESS	(472, 486, 'VEVTEKMTTVLSESS')
1	15	EVTEKMTTVLSESSGPSN	(473, 490, 'EVTEKMTTVLSESSGPSN')
1	15	VTEKMTTVLSESS	(474, 486, 'VTEKMTTVLSESS')
1	15	T	(475, 475, 'T')
1	15	EKMTTVLSESSGPSNA	(476, 491, 'EKMTTVLSESSGPSNA')
1	15	K	(477, 477, 'K')
1	15	T	(480, 480, 'T')
1	15	PSNASHS	(488, 494, 'PSNASHS')
1	15	SRESHLSNGLDPVAETAI	(496, 513, 'SRESHLSNGLDPVAETAI')
1	15	T	(511, 511, 'T')
1	15	RQLTEPSGRAAKKTP	(514, 528, 'RQLTEPSGRAAKKTP')
1	15	T	(517, 517, 'T')
1	15	K	(526, 526, 'K')
1	15	T	(527, 527, 'T')
1	15	T	(529, 529, 'T')
1	15	K	(530, 530, 'K')
1	15	T	(533, 533, 'T')
1	15	K	(541, 541, 'K')
1	15	DEL	(547, 549, 'DEL')
1	15	EL	(548, 549, 'EL')
1	15	YA	(555, 556, 'YA')
1	15	LE	(559, 560,

1	8	WAALVQAA	(51, 58, 'WAALVQAA')
1	8	VQAADT	(55, 60, 'VQAADT')
1	8	AADTLN	(57, 62, 'AADTLN')
1	8	T	(60, 60, 'T')
1	8	K	(68, 68, 'K')
1	8	T	(71, 71, 'T')
1	8	T	(79, 79, 'T')
1	8	FL	(80, 81, 'FL')
1	8	K	(83, 83, 'K')
1	8	LE	(89, 90, 'LE')
1	8	VLEGTLQCPE	(93, 102, 'VLEGTLQCPE')
1	8	LE	(94, 95, 'LE')
1	8	T	(97, 97, 'T')
1	8	LQCPESG	(98, 104, 'LQCPESG')
1	8	PESGRLFPISRGIPN	(101, 115, 'PESGRLFPISRGIPN')
1	8	FPI	(107, 109, 'FPI')
1	8	NMLLN	(115, 119, 'NMLLN')
1	8	MLL	(116, 118, 'MLL')
1	8	T	(123, 123, 'T')
1	8	T	(125, 125, 'T')
1	7	T	(3, 3, 'T')
1	7	QDAEAA	(15, 20, 'QDAEAA')
1	7	AINRQIN	(20, 26, 'AINRQIN')
1	7	EL	(28, 29, 'EL')
1	7	LYASYV	(29, 34, 'LYASYV')
1	7	YA	(30, 31, 'YA')
1	7	ASYVYL	(31, 36, 'ASYVYL')
1	7	DDVALKNFAKYFLHQS	(45, 60, 'DDVALKNFAKYFLHQS')
1	7	K	(54, 54, 'K')
1	7	YFLHQSHEE	(55, 63, 'YFLHQSHEE')
1	7	HEERE	(61, 65, 'HEERE')
1	7	EEREH	(62, 66, 'EEREH')
1	7	K	(69, 69, 'K')
1	7	K	(72, 72, 'K')
1	7	LQNQRG	(73, 78, 'LQNQRG')
1	7	QRGGRIFLQD	(76, 85, 'QRGGRIFLQD')
1	7	GRIFLQ	(79, 84

# Clustering y_hits

In [8]:
file_name = 'y_hits.txt'
ion = 'y'
# file_name = sys.argv[1]

hits = []

for l in open(file_name):
    A = l.rstrip().split('\t')
    pid = int(A[2])
    start = int(A[4].split('-')[0])
    end = int(A[4].split('-')[1])
    seq = A[3]

    hits.append( Hit(pid=pid, start=start, end=end, seq=seq) )

sorted_hits = sorted(hits, key=operator.attrgetter('pid', 'start', 'end'))

last_pid = None
last_start = None

cluster = []
to_print = []

with open('clusters.txt', 'w') as c:
    c.write('')

if ion == 'y':
    for hit in sorted_hits:
        if last_pid == hit.pid and last_start == hit.start:
            cluster.append(hit)
        else:
            write_cluster(cluster)
            cluster = [hit]
        last_pid = hit.pid
        last_start = hit.start

# Printing data

In [9]:
# y_hits
y_cluster_array = []
cluster = collections.namedtuple('cluster', 'score pid seq indices')
with open('clusters.txt', 'r') as c:
    for line in c:
        A = line.rstrip().split('\t')
        score = int(A[0])
        pid = int(A[1])
        seq = A[2]
        indices = []
        [indices.append(A[x]) for x in range(3,len(A))]

        y_cluster_array.append(cluster(score=score, pid=pid, seq=seq, indices=indices) )

y_sorted_clusters = sorted(y_cluster_array, key=operator.attrgetter('score', 'pid'), reverse = True)
for cluster in y_sorted_clusters:
    non_indices = str(cluster.score) + '\t' + str(cluster.pid) + '\t' + cluster.seq
    print(non_indices + '\t'+ '\t'.join([str(o) for o in cluster.indices]))
    

5	155	AGHGQEVL	(23, 26, 'AGHG')	(23, 27, 'AGHGQ')	(23, 29, 'AGHGQEV')	(23, 29, 'AGHGQEV')	(23, 30, 'AGHGQEVL')
5	76	PHSELDLE	(1410, 1413, 'PHSE')	(1410, 1415, 'PHSELD')	(1410, 1415, 'PHSELD')	(1410, 1416, 'PHSELDL')	(1410, 1417, 'PHSELDLE')
5	66	ICNDVLSLLEKFLI	(93, 93, 'I')	(93, 94, 'IC')	(93, 95, 'ICN')	(93, 104, 'ICNDVLSLLEKF')	(93, 106, 'ICNDVLSLLEKFLI')
4	275	LGSYFPKK	(29, 29, 'L')	(29, 33, 'LGSYF')	(29, 35, 'LGSYFPK')	(29, 36, 'LGSYFPKK')
4	261	TDAYLD	(66, 67, 'TD')	(66, 69, 'TDAY')	(66, 71, 'TDAYLD')	(66, 71, 'TDAYLD')
4	259	ESYGLEL	(62, 63, 'ES')	(62, 67, 'ESYGLE')	(62, 67, 'ESYGLE')	(62, 68, 'ESYGLEL')
4	253	LEPGADP	(29, 29, 'L')	(29, 30, 'LE')	(29, 31, 'LEP')	(29, 35, 'LEPGADP')
4	249	SFSEVEL	(214, 217, 'SFSE')	(214, 219, 'SFSEVE')	(214, 219, 'SFSEVE')	(214, 220, 'SFSEVEL')
4	246	LEPEEDFEQFL	(18, 18, 'L')	(18, 19, 'LE')	(18, 20, 'LEP')	(18, 28, 'LEPEEDFEQFL')
4	233	TVAYTEKKMT	(219, 225, 'TVAYTEK')	(219, 226, 'TVAYTEKK')	(219, 227, 'TVAYTEKKM')	(219, 228, 'TVAYTEKKMT')
4	233	IY

3	122	ICN	(95, 95, 'I')	(95, 96, 'IC')	(95, 97, 'ICN')
3	119	LEIVQAIYKMVS	(122, 122, 'L')	(122, 123, 'LE')	(122, 133, 'LEIVQAIYKMVS')
3	117	IQKHDGI	(76, 76, 'I')	(76, 81, 'IQKHDG')	(76, 82, 'IQKHDGI')
3	117	QKHDGITVAVHKMAS	(77, 79, 'QKH')	(77, 82, 'QKHDGI')	(77, 91, 'QKHDGITVAVHKMAS')
3	117	DGITVAVHKM	(80, 87, 'DGITVAVH')	(80, 88, 'DGITVAVHK')	(80, 89, 'DGITVAVHKM')
3	116	LEDVQAH	(334, 334, 'L')	(334, 335, 'LE')	(334, 340, 'LEDVQAH')
3	113	LEP	(46, 46, 'L')	(46, 47, 'LE')	(46, 48, 'LEP')
3	113	ICTL	(76, 76, 'I')	(76, 77, 'IC')	(76, 79, 'ICTL')
3	111	LEFGPENCTSWI	(472, 472, 'L')	(472, 473, 'LE')	(472, 483, 'LEFGPENCTSWI')
3	111	IERARA	(495, 495, 'I')	(495, 496, 'IE')	(495, 500, 'IERARA')
3	110	LLHHYVSVPQVSRSGAQ	(381, 381, 'L')	(381, 394, 'LLHHYVSVPQVSRS')	(381, 397, 'LLHHYVSVPQVSRSGAQ')
3	110	LTPCTVLPVKLYSDA	(734, 734, 'L')	(734, 743, 'LTPCTVLPVK')	(734, 748, 'LTPCTVLPVKLYSDA')
3	110	LVEFSC	(902, 902, 'L')	(902, 907, 'LVEFSC')	(902, 907, 'LVEFSC')
3	110	LEMSALF	(930, 930, 'L')	(930, 931

2	263	EIRHVLVTLGEK	(108, 109, 'EI')	(108, 119, 'EIRHVLVTLGEK')
2	262	RSYVTTSTRT	(28, 36, 'RSYVTTSTR')	(28, 37, 'RSYVTTSTRT')
2	262	TYSLGSAL	(37, 43, 'TYSLGSA')	(37, 44, 'TYSLGSAL')
2	262	PSTSRSLYSSS	(46, 47, 'PS')	(46, 56, 'PSTSRSLYSSS')
2	262	LYSS	(52, 52, 'L')	(52, 55, 'LYSS')
2	262	LRSSVPG	(70, 70, 'L')	(70, 76, 'LRSSVPG')
2	262	LLQDSVDFSLADA	(79, 79, 'L')	(79, 91, 'LLQDSVDFSLADA')
2	262	LQDSVDFSLADAI	(80, 80, 'L')	(80, 92, 'LQDSVDFSLADAI')
2	262	LE	(124, 124, 'L')	(124, 125, 'LE')
2	262	LE	(135, 135, 'L')	(135, 136, 'LE')
2	262	ESTLQSF	(200, 201, 'ES')	(200, 206, 'ESTLQSF')
2	262	TLQSFRQDVDNA	(202, 208, 'TLQSFRQ')	(202, 213, 'TLQSFRQDVDNA')
2	262	LE	(220, 220, 'L')	(220, 221, 'LE')
2	262	IQELQAQIQ	(242, 242, 'I')	(242, 250, 'IQELQAQIQ')
2	262	DVRQQYESVAAKNLQE	(271, 282, 'DVRQQYESVAAK')	(271, 286, 'DVRQQYESVAAKNLQE')
2	262	AEEWY	(287, 291, 'AEEWY')	(287, 291, 'AEEWY')
2	262	ESNEYRRQVQS	(315, 316, 'ES')	(315, 325, 'ESNEYRRQVQS')
2	262	LTCEVDALKGTNESLER	(326, 326, 'L')	(326, 342, 'LTC

2	220	LE	(150, 150, 'L')	(150, 151, 'LE')
2	220	HMALD	(162, 164, 'HMA')	(162, 166, 'HMALD')
2	219	LSMLSLIC	(15, 15, 'L')	(15, 22, 'LSMLSLIC')
2	219	LICSCFYPEP	(20, 20, 'L')	(20, 29, 'LICSCFYPEP')
2	219	IC	(21, 21, 'I')	(21, 22, 'IC')
2	219	ELILKPPSPISEA	(55, 56, 'EL')	(55, 67, 'ELILKPPSPISEA')
2	219	SPISEAPRTL	(62, 63, 'SP')	(62, 71, 'SPISEAPRTL')
2	219	LE	(81, 81, 'L')	(81, 82, 'LE')
2	219	LE	(88, 88, 'L')	(88, 89, 'LE')
2	219	LKQLAEKREHEREVL	(103, 103, 'L')	(103, 117, 'LKQLAEKREHEREVL')
2	219	LE	(121, 121, 'L')	(121, 122, 'LE')
2	219	IE	(154, 154, 'I')	(154, 155, 'IE')
2	218	IAVIGDEDTVTGFLLG	(8, 8, 'I')	(8, 23, 'IAVIGDEDTVTGFLLG')
2	218	IGDEDTVTGFLLGG	(11, 11, 'I')	(11, 24, 'IGDEDTVTGFLLGG')
2	218	DEDTVTGFLLGGIGELNK	(13, 26, 'DEDTVTGFLLGGIG')	(13, 30, 'DEDTVTGFLLGGIGELNK')
2	218	LGGIGELNKNRHPNFLV	(22, 22, 'L')	(22, 38, 'LGGIGELNKNRHPNFLV')
2	218	NKNRHPN	(29, 30, 'NK')	(29, 35, 'NKNRHPN')
2	218	IE	(48, 48, 'I')	(48, 49, 'IE')
2	218	LDAHQRSI	(78, 78, 'L')	(78, 85, 'LDAHQRSI')
2	218	LE	

2	191	IE	(920, 920, 'I')	(920, 921, 'IE')
2	191	LE	(938, 938, 'L')	(938, 939, 'LE')
2	191	ISHFMQFPHHL	(977, 977, 'I')	(977, 987, 'ISHFMQFPHHL')
2	191	IE	(991, 991, 'I')	(991, 992, 'IE')
2	191	PLAGQVNTMVTTSTTT	(1030, 1034, 'PLAGQ')	(1030, 1045, 'PLAGQVNTMVTTSTTT')
2	191	LSQSNMTQKVEE	(1105, 1105, 'L')	(1105, 1116, 'LSQSNMTQKVEE')
2	191	KNP	(1156, 1157, 'KN')	(1156, 1158, 'KNP')
2	191	LGMITLAKNKPILHTDL	(1198, 1198, 'L')	(1198, 1214, 'LGMITLAKNKPILHTDL')
2	191	NKP	(1206, 1207, 'NK')	(1206, 1208, 'NKP')
2	191	LE	(1221, 1221, 'L')	(1221, 1222, 'LE')
2	191	LE	(1242, 1242, 'L')	(1242, 1243, 'LE')
2	191	LVFRPPNP	(1249, 1249, 'L')	(1249, 1256, 'LVFRPPNP')
2	191	IE	(1282, 1282, 'I')	(1282, 1283, 'IE')
2	191	LC	(1285, 1285, 'L')	(1285, 1286, 'LC')
2	191	DVKQPE	(1321, 1325, 'DVKQP')	(1321, 1326, 'DVKQPE')
2	191	ELP	(1327, 1328, 'EL')	(1327, 1329, 'ELP')
2	191	QYSYH	(1355, 1359, 'QYSYH')	(1355, 1359, 'QYSYH')
2	191	IPLFQAHPQLKQCV	(1379, 1379, 'I')	(1379, 1392, 'IPLFQAHPQLKQCV')
2	191	IKIAMTTCEQ	(141

...

1	27	TAEHTERSLQ	(2853, 2862, 'TAEHTERSLQ')
1	27	EHTERSLQA	(2855, 2863, 'EHTERSLQA')
1	27	TD	(2868, 2869, 'TD')
1	27	L	(2870, 2870, 'L')
1	27	SDLSLEEERSLSIPSSE	(2887, 2903, 'SDLSLEEERSLSIPSSE')
1	27	DLSLEEERSLSIPSSES	(2888, 2904, 'DLSLEEERSLSIPSSES')
1	27	L	(2889, 2889, 'L')
1	27	L	(2897, 2897, 'L')
1	27	I	(2899, 2899, 'I')
1	27	PS	(2900, 2901, 'PS')
1	27	SE	(2902, 2903, 'SE')
1	27	ES	(2903, 2904, 'ES')
1	27	SE	(2904, 2905, 'SE')
1	27	L	(2918, 2918, 'L')
1	27	SE	(2924, 2925, 'SE')
1	27	L	(2927, 2927, 'L')
1	27	L	(2928, 2928, 'L')
1	27	DVDGNDLLSYWP	(2933, 2944, 'DVDGNDLLSYWP')
1	27	L	(2940, 2940, 'L')
1	27	SYWPALGECEAAP	(2941, 2953, 'SYWPALGECEAAP')
1	27	L	(2946, 2946, 'L')
1	27	L	(2956, 2956, 'L')
1	27	WGS	(2959, 2961, 'WGS')
1	27	SE	(2961, 2962, 'SE')
1	27	L	(2965, 2965, 'L')
1	27	L	(2967, 2967, 'L')
1	27	NK	(2970, 2971, 'NK')
1	27	NQP	(2977, 2979, 'NQP')
1	27	PEL	(2979, 2981, 'PEL')
1	27	EL	(2980, 2981, 'EL')
1	27	L	(2981, 2981, 'L')
1	27	L	(2983, 2983, 'L')
1	27	L	(2991, 2991, 'L')
1

1	20	TEAHE	(38, 42, 'TEAHE')
1	20	EAHEDP	(39, 44, 'EAHEDP')
1	20	NEDKRHSQGT	(48, 57, 'NEDKRHSQGT')
1	20	FTSD	(58, 61, 'FTSD')
1	20	L	(66, 66, 'L')
1	20	SRRAQDF	(68, 74, 'SRRAQDF')
1	20	AQDFVQWLM	(71, 79, 'AQDFVQWLM')
1	20	FVQWLM	(74, 79, 'FVQWLM')
1	20	L	(78, 78, 'L')
1	20	NTKRNRNNIAKRH	(80, 92, 'NTKRNRNNIAKRH')
1	20	RNRNNIAKRHD	(83, 93, 'RNRNNIAKRHD')
1	20	RNNIAK	(85, 90, 'RNNIAK')
1	20	NNIAKR	(86, 91, 'NNIAKR')
1	20	I	(88, 88, 'I')
1	20	FER	(95, 97, 'FER')
1	20	ERHAEG	(96, 101, 'ERHAEG')
1	20	FTSD	(103, 106, 'FTSD')
1	20	VSSYLE	(107, 112, 'VSSYLE')
1	20	SSYL	(108, 111, 'SSYL')
1	20	YLEGQAAK	(110, 117, 'YLEGQAAK')
1	20	EGQAAKEF	(112, 119, 'EGQAAKEF')
1	20	I	(120, 120, 'I')
1	20	L	(123, 123, 'L')
1	20	I	(138, 138, 'I')
1	20	EL	(141, 142, 'EL')
1	20	L	(142, 142, 'L')
1	20	SFSDEM	(150, 155, 'SFSDEM')
1	20	FSDEMS	(151, 156, 'FSDEMS')
1	20	MSTI	(155, 158, 'MSTI')
1	20	I	(158, 158, 'I')
1	20	L	(159, 159, 'L')
1	20	L	(162, 162, 'L')
1	20	FINWLIQTKITDK	(167, 179, 'FINWLIQTKITDK')
1	20	I	(168,

1	15	L	(342, 342, 'L')
1	15	EL	(348, 349, 'EL')
1	15	L	(349, 349, 'L')
1	15	L	(351, 351, 'L')
1	15	L	(365, 365, 'L')
1	15	GQATLP	(366, 371, 'GQATLP')
1	15	L	(370, 370, 'L')
1	15	SP	(374, 375, 'SP')
1	15	PS	(375, 376, 'PS')
1	15	MSRRQVC	(379, 385, 'MSRRQVC')
1	15	RQVCPL	(382, 387, 'RQVCPL')
1	15	QVC	(383, 385, 'QVC')
1	15	PLTPGPGKSLSPAAT	(386, 400, 'PLTPGPGKSLSPAAT')
1	15	L	(387, 387, 'L')
1	15	GPGK	(390, 393, 'GPGK')
1	15	L	(395, 395, 'L')
1	15	SP	(396, 397, 'SP')
1	15	AELH	(403, 406, 'AELH')
1	15	EL	(404, 405, 'EL')
1	15	L	(405, 405, 'L')
1	15	L	(415, 415, 'L')
1	15	PS	(425, 426, 'PS')
1	15	I	(427, 427, 'I')
1	15	PTKKIE	(429, 434, 'PTKKIE')
1	15	EL	(434, 435, 'EL')
1	15	L	(435, 435, 'L')
1	15	I	(439, 439, 'I')
1	15	VQSRP	(452, 456, 'VQSRP')
1	15	L	(462, 462, 'L')
1	15	DSPSRSPSKVEVTEKMT	(463, 479, 'DSPSRSPSKVEVTEKMT')
1	15	SP	(464, 465, 'SP')
1	15	PS	(465, 466, 'PS')
1	15	SP	(468, 469, 'SP')
1	15	PS	(469, 470, 'PS')
1	15	MTTV	(478, 481, 'MTTV')
1	15	L	(482, 482, 'L')
1	15	SE	(483, 484,

1	13	SE	(809, 810, 'SE')
1	13	DT	(815, 816, 'DT')
1	13	L	(819, 819, 'L')
1	13	TAEDLINACSHYG	(821, 833, 'TAEDLINACSHYG')
1	13	L	(825, 825, 'L')
1	13	I	(826, 826, 'I')
1	13	L	(834, 834, 'L')
1	13	L	(842, 842, 'L')
1	13	SLADKNY	(847, 853, 'SLADKNY')
1	13	KN	(851, 852, 'KN')
1	13	TD	(854, 855, 'TD')
1	13	L	(857, 857, 'L')
1	13	L	(858, 858, 'L')
1	13	L	(860, 860, 'L')
1	13	L	(863, 863, 'L')
1	13	SP	(866, 867, 'SP')
1	13	PS	(867, 868, 'PS')
1	13	L	(869, 869, 'L')
1	13	I	(874, 874, 'I')
1	13	PFLEPL	(876, 881, 'PFLEPL')
1	13	EPL	(879, 881, 'EPL')
1	13	L	(881, 881, 'L')
1	13	SE	(882, 883, 'SE')
1	13	DT	(884, 885, 'DT')
1	13	L	(889, 889, 'L')
1	13	THALCH	(891, 896, 'THALCH')
1	13	HALCHT	(892, 897, 'HALCHT')
1	13	ALCHTRLQE	(893, 901, 'ALCHTRLQE')
1	13	CI	(905, 906, 'CI')
1	13	I	(906, 906, 'I')
1	13	DT	(907, 908, 'DT')
1	13	L	(909, 909, 'L')
1	13	I	(918, 918, 'I')
1	13	EL	(924, 925, 'EL')
1	13	L	(925, 925, 'L')
1	13	I	(931, 931, 'I')
1	13	L	(932, 932, 'L')
1	13	PEL	(939, 941, 'PEL')
1	13	EL	(940, 

1	7	GLNAMECALHL	(97, 107, 'GLNAMECALHL')
1	7	L	(98, 98, 'L')
1	7	L	(115, 115, 'L')
1	7	EL	(117, 118, 'EL')
1	7	L	(118, 118, 'L')
1	7	L	(121, 121, 'L')
1	7	KN	(125, 126, 'KN')
1	7	PHLC	(128, 131, 'PHLC')
1	7	L	(139, 139, 'L')
1	7	SE	(140, 141, 'SE')
1	7	EL	(148, 149, 'EL')
1	7	L	(149, 149, 'L')
1	7	L	(156, 156, 'L')
1	7	AGMA	(164, 167, 'AGMA')
1	7	L	(170, 170, 'L')
1	7	L	(176, 176, 'L')
1	7	ES	(181, 182, 'ES')
1	6	PS	(6, 7, 'PS')
1	6	SYGLSRE	(7, 13, 'SYGLSRE')
1	6	YGLSREV	(8, 14, 'YGLSREV')
1	6	L	(10, 10, 'L')
1	6	YDADLEQILIQWI	(22, 34, 'YDADLEQILIQWI')
1	6	I	(29, 29, 'I')
1	6	L	(30, 30, 'L')
1	6	I	(31, 31, 'I')
1	6	QWITTQC	(32, 38, 'QWITTQC')
1	6	I	(34, 34, 'I')
1	6	QPGRE	(46, 50, 'QPGRE')
1	6	L	(56, 56, 'L')
1	6	L	(65, 65, 'L')
1	6	I	(66, 66, 'I')
1	6	L	(69, 69, 'L')
1	6	PEGQAPV	(71, 77, 'PEGQAPV')
1	6	I	(80, 80, 'I')
1	6	FKQMEQ	(87, 92, 'FKQMEQ')
1	6	QMEQISQFLQAA	(89, 100, 'QMEQISQFLQAA')
1	6	I	(93, 93, 'I')
1	6	L	(97, 97, 'L')
1	6	RYGINTTD	(102, 109, 'RYGINTTD')
1	6	YGINTTDIFQ	(103,

# To Filter data by parent prot

I want to be able to only view the b or y hits from a certain protein

In [10]:
# b_hits
target_pid = 274
b_target_clusters = []
for cluster in b_sorted_clusters:
    if cluster.pid == target_pid:
        b_target_clusters.append(cluster)

for cluster in b_target_clusters:
    non_indices = str(cluster.score) + '\t' + str(cluster.pid) + '\t' + cluster.seq
    print(non_indices + '\t'+ '\t'.join([str(o) for o in cluster.indices]))


7	274	DPQVEQLE	(60, 62, 'DPQ')	(60, 63, 'DPQV')	(60, 64, 'DPQVE')	(60, 65, 'DPQVEQ')	(60, 65, 'DPQVEQ')	(60, 66, 'DPQVEQL')	(60, 67, 'DPQVEQLE')
3	274	LGGSPGDL	(68, 72, 'LGGSP')	(68, 73, 'LGGSPG')	(68, 75, 'LGGSPGDL')
2	274	FLP	(7, 8, 'FL')	(7, 9, 'FLP')
2	274	EDPQVEQL	(59, 63, 'EDPQV')	(59, 66, 'EDPQVEQL')
2	274	LELGGSPGDL	(66, 67, 'LE')	(66, 75, 'LELGGSPGDL')
2	274	LEVARQ	(80, 81, 'LE')	(80, 85, 'LEVARQ')
1	274	LLALWEP	(13, 19, 'LLALWEP')
1	274	K	(20, 20, 'K')
1	274	T	(22, 22, 'T')
1	274	K	(27, 27, 'K')
1	274	PH	(33, 34, 'PH')
1	274	HLVEALYL	(34, 41, 'HLVEALYL')
1	274	LVCGERG	(41, 47, 'LVCGERG')
1	274	FYTPKSRREVE	(49, 59, 'FYTPKSRREVE')
1	274	T	(51, 51, 'T')
1	274	K	(53, 53, 'K')
1	274	EVE	(57, 59, 'EVE')
1	274	VEDPQ	(58, 62, 'VEDPQ')
1	274	PQ	(61, 62, 'PQ')
1	274	EL	(67, 68, 'EL')
1	274	LQTLALEVA	(75, 83, 'LQTLALEVA')
1	274	T	(77, 77, 'T')
1	274	K	(86, 86, 'K')
1	274	T	(95, 95, 'T')
1	274	LE	(103, 104, 'LE')


In [11]:
# y_hits
target_pid = 274
y_target_clusters = []
for cluster in y_sorted_clusters:
    if cluster.pid == target_pid:
        y_target_clusters.append(cluster)

for cluster in y_target_clusters:
    non_indices = str(cluster.score) + '\t' + str(cluster.pid) + '\t' + cluster.seq
    print(non_indices + '\t'+ '\t'.join([str(o) for o in cluster.indices]))

3	274	LEVARQ	(80, 80, 'L')	(80, 81, 'LE')	(80, 85, 'LEVARQ')
2	274	LC	(30, 30, 'L')	(30, 31, 'LC')
2	274	DPQVEQLEL	(60, 65, 'DPQVEQ')	(60, 68, 'DPQVEQLEL')
2	274	LE	(66, 66, 'L')	(66, 67, 'LE')
2	274	LGGSPGDL	(68, 68, 'L')	(68, 75, 'LGGSPGDL')
2	274	IVDQCCTSICS	(89, 89, 'I')	(89, 99, 'IVDQCCTSICS')
2	274	IC	(97, 97, 'I')	(97, 98, 'IC')
2	274	LE	(103, 103, 'L')	(103, 104, 'LE')
1	274	L	(3, 3, 'L')
1	274	L	(4, 4, 'L')
1	274	L	(8, 8, 'L')
1	274	L	(10, 10, 'L')
1	274	L	(11, 11, 'L')
1	274	L	(13, 13, 'L')
1	274	L	(14, 14, 'L')
1	274	L	(16, 16, 'L')
1	274	PKPTQAFVKQHL	(19, 30, 'PKPTQAFVKQHL')
1	274	TQAFVKQHLCGPHLVEA	(22, 38, 'TQAFVKQHLCGPHLVEA')
1	274	FVKQ	(25, 28, 'FVKQ')
1	274	KQH	(27, 29, 'KQH')
1	274	L	(35, 35, 'L')
1	274	EALYLVC	(37, 43, 'EALYLVC')
1	274	L	(39, 39, 'L')
1	274	L	(41, 41, 'L')
1	274	EL	(67, 68, 'EL')
1	274	SP	(71, 72, 'SP')
1	274	L	(75, 75, 'L')
1	274	L	(78, 78, 'L')
1	274	ALEVARQKR	(79, 87, 'ALEVARQKR')
1	274	ARQKRG	(83, 88, 'ARQKRG')
1	274	VDQCCTSICSL	(90, 100, 'VDQCCTS

# See what non-hybrid solutions we can create

In [12]:
# b side starting. Ideally, we would probably pick the higher scoring side to start
filtered_b, filtered_y = testing_utils.get_top_X(b_sorted_clusters, y_sorted_clusters, 50)

# Calculating start and end indices for each interval
for b_cluster in filtered_b:
    interesting_combos = []
    b_index_set = set()
    for index in b_cluster.indices:
        index = index.replace('(', '')
        index = index.replace(')', '')
        A = index.rstrip().split(',')
        b_index_set.add(int(A[0]))
        b_index_set.add(int(A[1]))
    
    b_start_pos = min(b_index_set)
    b_end_pos = max(b_index_set)
    
    for y_cluster in filtered_y:
        y_index_set = set()
        for index in y_cluster.indices:
            index = index.replace('(', '')
            index = index.replace(')', '')
            A = index.rstrip().split(',')
            y_index_set.add(int(A[0]))
            y_index_set.add(int(A[1]))
            
        y_start_pos = min(y_index_set)
        y_end_pos = max(y_index_set)
        
        if (b_start_pos <= y_start_pos) and (b_end_pos == y_start_pos - 1):
            interesting_combos.append(b_cluster)
            interesting_combos.append(y_cluster)
    
    print(interesting_combos)

AttributeError: 'int' object has no attribute 'indices'